<a href="https://colab.research.google.com/github/emgabrielly/cybersecurity-check/blob/main/modelointerpolmng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q streamlit_ace
!pip install fuzzywuzzy[speedup]
!streamlit run app.py &>/dev/null&

In [ ]:
from fuzzywuzzy import process
from pyngrok import ngrok
from fuzzywuzzy import fuzz

In [ ]:
%%writefile app.py
import streamlit as st
import re
import requests as req

Overwriting app.py


In [ ]:
def verifpassaporte():
    lista_passaporte = []
    lista_arg = req.get("https://emgabrielly.github.io/mng.json")
    arg = lista_arg.json()['mng']
    for i in arg:
        lista_passaporte.append(str(i['mng']))
    return lista_passaporte
    
def verifnome():
  lista_nome = []
  lista_interpol = req.get("https://cspinheiro.github.io/interpol.json")
  interpol = lista_interpol.json()['interpol']
  for i in interpol:
    lista_nome.append(i['interpol'])
  return lista_nome

def regexlist(lista):
    a='(?i)'
    for i in lista:
        a += f"({i})|"
    a = a[:-1]
    return a

In [ ]:
def getmatch_nome(input,lista):
    search = process.extractOne(input,lista,scorer=fuzz.token_sort_ratio)
    if search[1] > 80:
      return 1
    return 0
        

def valid_nome(input_nome,lista_nome):
  if len(input_nome) > 3 and len(re.split('\s',input_nome)) >= 2:
      nome = getmatch_nome(input_nome,lista_nome)
      if nome == 1:
        st.error('Liberação do passageiro não autorizada!')
      if nome == 0:
        st.success('Bem-vindo, faça uma ótima viagem')
  else:
      st.warning('O Nome inserido é Invalido!')

def getmatch_passa(input,lista):
    match = re.search(lista,input)

    if match:
        return 1
    else:
        return 0

In [ ]:
def main():
    html_temp = """ <div style ="background-color:black;padding:13px">
                      <h1 style = "color:white;text-align:center;">Mongólia Aeroport - VSCT/h1>
                      <image src="bandeira-mongolia.jpg";align-objet:center;>
                    </dic>
                """

    st.markdown(html_temp,unsafe_allow_html = True)
    lista_passaporte = verifpassaporte()
    lista_nome = verifnome()

    lista_passaporte= regexlist(lista_passaporte)

    input_passaporte = st.text_input('Passaporte: ')
    input_nome = st.text_input('Nome Completo: ')

    input_passaporte = re.sub('\D', '', input_passaporte)

    if st.button("Verificar"):
      clear = 0
      clear = valid_passa(input_passaporte,lista_passaporte)
      if clear == 1:
        valid_nome(input_nome,lista_nome)

def valid_passa(input_passaporte,lista_passaporte):
      if len(input_passaporte) == 9:
        passa = getmatch_passa(input_passaporte,lista_passaporte)
        if passa == 1:
            st.error('Atenção! Passageiro apresenta risco, acione a segurança imediatamente')
            return 0
        else:
          return 1
      else:
        st.warning('Passaporte Invalido!')

if __name__ == '__main__':
    main()

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501